![title](wcar.png)
# Interaction Partners for PIG-H
    Zhe Ji, Michele Tinti and Michael A.J. Ferguson

    Wellcome Centre for Anti-Infectives Research
    School of Life Sciences, University of Dundee


In [2]:
#reload when modified
%load_ext autoreload
%autoreload 2

In [3]:
#set up code
%run imports.py 

In [4]:
import utilities
pd.set_option('max_colwidth', 12)

In [5]:
#create a description dictionary
#prot_id -> desc
path_to_file = 'TriTrypDB-41_TbruceiTREU927_AnnotatedProteins_plusVSG.fasta'
desc_dict = utilities.make_desc_dict(path_to_file)

In [6]:
df = pd.read_csv('data_iris_limma.csv.gz',index_col=[0])
df.head()

,Protein IDs,Majority protein IDs,Peptide counts (all),Peptide counts (razor+unique),Peptide counts (unique),Fasta headers,Number of proteins,Peptides,Razor + unique peptides,Unique peptides,...,edgeR_logCPM,edgeR_PValue,edgeR_FDR,edgeR_log10_PValue,limma_Acc,limma_PValue,limma_FDR,limma_candidate,limma_log10FDR,limma_log10PValue
155,Tb05.5K5...,Tb05.5K5...,97;97;55,97;97;55,97;97;55,Tb05.5K5...,3,97,97,97,...,12.752955,0.812829,1.0,0.090001,155,0.173880,0.567426,no,0.246091,0.759750
156,Tb927.5....,Tb927.5....,6;6,6;6,6;6,Tb927.5....,2,6,6,6,...,7.146750,1.000000,1.0,-0.000000,156,0.553512,0.753821,no,0.122732,0.256873
157,Tb927.5....,Tb927.5....,12;12,12;12,12;12,Tb927.5....,2,12,12,12,...,8.801039,1.000000,1.0,-0.000000,157,0.779904,0.893589,no,0.048862,0.107959
158,Tb927.5....,Tb927.5....,119;119,119;119,119;119,Tb927.5....,2,119,119,119,...,11.972702,0.901682,1.0,0.044947,158,0.356883,0.642493,no,0.192131,0.447474
159,Tb927.5....,Tb927.5....,18;18;18,18;18;18,18;18;18,Tb927.5....,3,18,18,18,...,9.198063,0.562645,1.0,0.249765,159,0.057613,0.533021,no,0.273256,1.239478


In [8]:
#list(df.columns)

In [31]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)


dataset = df.copy()

naming_dict={
'Tb11.v5.0765': 'TbPIGA',
'Tb927.5.3680': 'TbPIGH',
'Tb927.10.10110': 'TbPIGP',
'Tb927.10.6140': 'TbPIGC',
'Tb927.3.4570': 'TbPIGQ',
'Tb927.4.780': 'TbPIGY',
'Tb927.10.4610': 'DPM1',
'Tb927.9.6440': 'DPM2'}

gd = dataset[dataset['Gene_id'].isin(naming_dict.keys())]
dataset = dataset.iloc[~dataset.index.isin(gd.index.values)]

up = dataset[(dataset['log2FC']>4)]
down = dataset[ (dataset['log2FC']<-4)]

the_rest = dataset.iloc[~dataset.index.isin( list(up.index.values) + list(down .index.values) ) ]
gd[['Gene_id','Gene_desc','log2FC']]

,Gene_id,Gene_desc,log2FC
236,Tb927.10...,phosphat...,9.146524
360,Tb11.v5....,N-acetyl...,15.084654
506,Tb927.10...,"PIG-P, p...",13.161237
922,Tb927.10...,dolichol...,0.444158
2229,Tb927.3....,N-acetyl...,9.239462
2523,Tb927.4.780,Phosphat...,7.435387
2653,Tb927.5....,GPI-GlcN...,13.713447
3836,Tb927.9....,dolichol...,0.293483


In [28]:

#interactive plot 'logFC',y='log10p'
trace_1 = go.Scatter(
    x=the_rest.log2FC,
    y=the_rest['log10pvalue'],
    mode='markers',
    name='all',
    #text = [a + ' ' +b for a,b in zip(temp.index.values, temp.Gene_desc)],
    #hoverinfo = 'text',
    marker=dict(
        #size=16,
        color = 'aquamarine'#np.log10(temp.Score), #set color equal to a variable
        #colorscale='Bluered',
        #showscale=True
    )
)

trace_2 = go.Scatter(
    x=up.log2FC,
    y=up['log10pvalue'],
    mode='markers',
    name='Up',
    text = up.Gene_id.values +' '+up.Gene_desc,
    hoverinfo = 'text',
    marker=dict(
        #size=16,
        color = 'green'#np.log10(temp.Score), #set color equal to a variable
        #colorscale='Bluered',
        #showscale=True
    )
)

trace_3 = go.Scatter(
    x=down.log2FC,
    y=down['log10pvalue'],
    mode='markers',
    name='Down',
    text = down.Gene_id.values +' '+down.Gene_desc,
    hoverinfo = 'text',
    marker=dict(
        #size=16,
        color = 'blue'#np.log10(temp.Score), #set color equal to a variable
        #colorscale='Bluered',
        #showscale=True
    )
)


trace_4 = go.Scatter(
    x=gd.log2FC,
    y=gd['log10pvalue'],
    mode='markers',
    name='GD',
    text = [naming_dict[n] for n in gd['Gene_id']],
    hoverinfo = 'text',
    
    marker=dict(
        #size=16,
        color = 'red'#np.log10(temp.Score), #set color equal to a variable
        #colorscale='Bluered',
        #showscale=True
    )
)




layout = dict(title='Vulcano',
             width=1000,
             height=1000)

fig = go.Figure(data=[trace_1,trace_2,trace_3,trace_4], 
                layout=layout)

iplot(fig, filename='test')